In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext
sc=SparkContext(master="local[3]")
print(sc)
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark_airport").getOrCreate()

<SparkContext master=local[3] appName=pyspark-shell>


In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.regression import GBTRegressionModel, GBTRegressor

from pyspark.ml.feature import VectorAssembler


In [3]:
midata = spark.read.csv('./train.csv', header = 'true', inferSchema = 'true', sep = ','  )   

In [4]:
midata.take(1)

[Row(Id=1, MSSubClass=60, MSZoning='RL', LotFrontage='65', LotArea=8450, Street='Pave', Alley='NA', LotShape='Reg', LandContour='Lvl', Utilities='AllPub', LotConfig='Inside', LandSlope='Gtl', Neighborhood='CollgCr', Condition1='Norm', Condition2='Norm', BldgType='1Fam', HouseStyle='2Story', OverallQual=7, OverallCond=5, YearBuilt=2003, YearRemodAdd=2003, RoofStyle='Gable', RoofMatl='CompShg', Exterior1st='VinylSd', Exterior2nd='VinylSd', MasVnrType='BrkFace', MasVnrArea='196', ExterQual='Gd', ExterCond='TA', Foundation='PConc', BsmtQual='Gd', BsmtCond='TA', BsmtExposure='No', BsmtFinType1='GLQ', BsmtFinSF1=706, BsmtFinType2='Unf', BsmtFinSF2=0, BsmtUnfSF=150, TotalBsmtSF=856, Heating='GasA', HeatingQC='Ex', CentralAir='Y', Electrical='SBrkr', 1stFlrSF=856, 2ndFlrSF=854, LowQualFinSF=0, GrLivArea=1710, BsmtFullBath=1, BsmtHalfBath=0, FullBath=2, HalfBath=1, BedroomAbvGr=3, KitchenAbvGr=1, KitchenQual='Gd', TotRmsAbvGrd=8, Functional='Typ', Fireplaces=0, FireplaceQu='NA', GarageType='Att

Hacemos un cast de nuestras variables a Double

In [5]:
import pyspark.ml.feature as ft
import pyspark.sql.types as typ
midata = midata.withColumn('YearBuilt', midata['YearBuilt'].cast(typ.DoubleType()))
midata = midata.withColumn('SalePrice', midata['SalePrice'].cast(typ.DoubleType()))
midata = midata.withColumn('TotalBsmtSF', midata['TotalBsmtSF'].cast(typ.DoubleType()))
midata = midata.withColumn('OverallQual', midata['OverallQual'].cast(typ.DoubleType()))
midata = midata.withColumn('1stFlrSF', midata['1stFlrSF'].cast(typ.DoubleType()))
midata = midata.withColumn('GrLivArea', midata['GrLivArea'].cast(typ.DoubleType()))
midata = midata.withColumn('GarageArea', midata['GarageArea'].cast(typ.DoubleType()))

In [6]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer  

In [7]:
featuresCreator = VectorAssembler(inputCols=["YearBuilt","TotalBsmtSF","OverallQual","1stFlrSF","GrLivArea","GarageArea", ],outputCol='features')

In [8]:
train_data,test_data=midata.randomSplit([0.80,0.20])

In [9]:
[{'maxDepth': 4}, {'maxBins': 60}, {'maxIter': 10}]

[{'maxDepth': 4}, {'maxBins': 60}, {'maxIter': 10}]

In [10]:
gtb =GBTRegressor(labelCol='SalePrice' ,featuresCol='features', maxDepth = 4,maxBins = 60 ,maxIter= 100 )

In [11]:
pipeline = Pipeline(stages=[featuresCreator, gtb])

In [12]:
model = pipeline.fit(train_data)

In [13]:
predictions = model.transform(train_data)

In [14]:
predictions.select("prediction", "SalePrice", "features").show()

+------------------+---------+--------------------+
|        prediction|SalePrice|            features|
+------------------+---------+--------------------+
| 197404.2659384325| 208500.0|[2003.0,856.0,7.0...|
| 165263.4208222521| 181500.0|[1976.0,1262.0,6....|
|196198.05193780814| 223500.0|[2001.0,920.0,7.0...|
|  145870.124430958| 140000.0|[1915.0,756.0,7.0...|
|271415.08990951424| 250000.0|[2000.0,1145.0,8....|
| 144797.8909999396| 143000.0|[1993.0,796.0,5.0...|
|281659.42023441504| 307000.0|[2004.0,1686.0,8....|
|203661.43773385405| 200000.0|[1973.0,1107.0,7....|
|162930.28009950148| 129900.0|[1931.0,952.0,7.0...|
|137010.99585081614| 129500.0|[1965.0,1040.0,5....|
| 330941.0319500822| 345000.0|[2005.0,1175.0,9....|
| 250845.4367056479| 279500.0|[2006.0,1494.0,7....|
| 141911.5673511053| 157000.0|[1960.0,1253.0,6....|
|140521.99025411913| 132000.0|[1929.0,832.0,7.0...|
| 94338.32879194601|  90000.0|[1967.0,0.0,4.0,1...|
|160740.59839145103| 159000.0|[2004.0,1114.0,5....|
| 147938.928

In [17]:
evaluator = RegressionEvaluator(labelCol='SalePrice', predictionCol='prediction',metricName = 'rmse')

Error cuadratico medio mide el promedio de los errores al cuadrado, es decir, la diferencia entre el estimador y lo que se estima

In [18]:
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
print("MSE: %.3f" % rmse)

MSE: 14093.753


El coeficiente de determinacion determina la calidad del modelo para Predecir los resultados

In [19]:
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.965


In [17]:
import pyspark.ml.regression as rg
from pyspark.ml.regression import GBTRegressor
import pyspark.ml.tuning as tune

In [18]:
boot = rg.GBTRegressor(labelCol = 'SalePrice')
grid = tune.ParamGridBuilder().addGrid(boot.maxDepth, [2, 4, 6]).addGrid(boot.maxBins, [20, 60]).addGrid(boot.maxIter, [10, 20]).build()

In [19]:
evaluatorG = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'SalePrice')

CROSS VALIDATION

In [20]:
gbt1 = tune.CrossValidator(estimator = boot, estimatorParamMaps = grid, evaluator = evaluatorG)

In [21]:
pipeline = Pipeline(stages=[featuresCreator])
data_transformer = pipeline.fit(train_data)

In [22]:
rgModel = gbt1.fit(data_transformer.transform(train_data))

In [23]:
data_train = data_transformer.transform(test_data)
results = rgModel.transform(data_train)

print(evaluator.evaluate(results, {evaluatorG.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(results, {evaluatorG.metricName: 'areaUnderPR'}))

NameError: name 'evaluator' is not defined

In [ ]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        rgModel.getEstimatorParamMaps(), 
        rgModel.avgMetrics
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

EXTRACCIÓN DE CARACTERÍSTICAS - PCA